### Chebyshev

The generic Chebyshev approximation problem is (for $i = 1,...,k$)
\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \max_i \left|b_i^Tx - c_i\right| \\
  \end{aligned}
\end{equation*}

We add an L2 term to form the problem:

\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \max_i \| A^{(i)}x \|_2 + \left|b_i^Tx - c_i\right| \\
  \end{aligned}
\end{equation*}

or equivalently:

\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \max_i t + \left|b_i^Tx - c_i\right| \\
    &\text{subject to} && \|A^{(i)}x\|_2 \leq t_i &&&i = 1,...,k
  \end{aligned}
\end{equation*}

In [2]:
import cvxpy as cp
import numpy as np
import scipy as sp

# Variable declarations

def normalized_data_matrix(m, n, mu):
    if mu == 1:
        # dense
        A = np.random.randn(m, n)
        A /= np.sqrt(np.sum(A**2, 0))
    else:
        # sparse
        A = sp.rand(m, n, mu)
        A.data = np.random.randn(A.nnz)
        N = A.copy()
        N.data = N.data**2
        A = A*sp.diags([1 / np.sqrt(np.ravel(N.sum(axis=0)))], [0])

    return A

np.random.seed(0)
m = 100
n = 200
k = 10
A = [normalized_data_matrix(m,n,1) for i in range(k)]
B = normalized_data_matrix(k,n,1)
c = np.random.rand(k)

x = cp.Variable(n)
t = cp.Variable(k)

In [8]:
# Problem construction
prob = None
opt_val = None


f = cp.max_entries(t+cp.abs(B*x-c))
C = []
for i in range(k):
    C.append(cp.pnorm(A[i]*x, 2) <= t[i])

prob = cp.Problem(cp.Minimize(f), C)

## Equivalent formulation:
obj_list = [cp.pnorm(A[i]*x, 2) + cp.abs(B[i,:]*x - c[i]) for i in range(k)]
f2 = cp.max_elemwise(obj_list)
prob2 = cp.Problem(cp.Minimize(f2))

# For debugging individual problems:
if __name__ == "__main__":
    prob.solve()
    prob2.solve()
    print("status:", prob.status)
    print("optimal value:", prob.value)
    print("true optimal value:", opt_val)
    print("status:", prob2.status)
    print("optimal value:", prob2.value)
    print("true optimal value:", opt_val)

('status:', 'optimal')
('optimal value:', 0.32590134030887485)
('true optimal value:', None)
('status:', 'optimal')
('optimal value:', 0.3259013412464839)
('true optimal value:', None)
